<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

total no of sub arrays which sum up to k.

In [ ]:
import java.util.ArrayList;
import java.util.HashMap;

public class totalSubarrayEqualToK {
    public static void main(String[] args){
        int[] arr = {1,-1,0};
        int k = 0;
        int count = 0;
        HashMap<Integer, Integer> prefixSumCount = new HashMap<>();
        int sum = 0;
        prefixSumCount.put(0, 1);
        for (int i = 0; i < arr.length; i++) {
            sum += arr[i];
            if (prefixSumCount.containsKey(sum - k)) {
                count += prefixSumCount.get(sum - k);
            }
            prefixSumCount.put(sum, prefixSumCount.getOrDefault(sum, 0) + 1);
        }
        System.out.println(count);
    }
}



Binary search

In [ ]:
public class binarySearch {
    public static void main(String[] args){
        int[] nums = {-1,0,3,5,9,12};
        int target = 9;
        int left = 0;
        int right = nums.length-1;
        while(left <= right){
            int mid = (left+right)/2;
            if(nums[mid] == target){
                System.out.println("Element found at index: "+mid);
                break;
            }
            else if(nums[mid]<target){
                left = mid +1;
            }
            else{
                right = mid-1;
            }
        }
        System.out.println("Element no found");
    }

}


floor value


In [ ]:
public class floorOfK {
    public static void main(String[] args) {
        int[] nums = {1,2,4,10,11,12,19};
        int left = 0;
        int right = nums.length-1;
        int target = 5;
        int floorindex =-1;
        while(left<=right){
            int mid = (left + right)/2;
            if(nums[mid]==target){
                System.out.println(mid);
                break;
            } else if (nums[mid]<target) {
                floorindex = mid;
                left = mid +1;
            }
            else{
                right = mid -1;
            }
        }
        System.out.println(floorindex);
    }
}
